## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [1]:
spark

In [2]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [4]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [5]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [6]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [7]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [8]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [9]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [10]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [11]:
result = run_sql('''
                    SELECT name, membercost FROM facilities WHERE membercost = 0
                   
                 ''')
result.toPandas()

,name,membercost
0,Badminton Court,0.0
1,Table Tennis,0.0
2,Snooker Table,0.0
3,Pool Table,0.0


####  Q2: How many facilities do not charge a fee to members?

In [12]:
result = run_sql('''
                   SELECT * FROM facilities WHERE membercost = 0
                 ''')
result.toPandas().count()

facid                 4
name                  4
membercost            4
guestcost             4
initialoutlay         4
monthlymaintenance    4
dtype: int64

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [13]:
result = run_sql('''
                   SELECT * FROM facilities
                   WHERE membercost / monthlymaintenance < 0.2
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [14]:
result = run_sql('''
                   SELECT * FROM facilities
                   WHERE facid IN (1,5)
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [15]:
result = run_sql('''
                   SELECT name, IF(monthlymaintenance<100, "cheap", "expensive") FROM facilities
                 ''')
result.toPandas()

,name,"(IF((monthlymaintenance < 100), cheap, expensive))"
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [16]:
result = run_sql('''
                    SELECT firstname, surname, joindate FROM members
                    WHERE joindate = (
                    SELECT MAX(joindate) from members
                    ) OR 
                    joindate = (
                    SELECT MIN(joindate) from members
                    )
                 ''')
result.toPandas()

,firstname,surname,joindate
0,GUEST,GUEST,2012-07-01 00:00:00
1,Darren,Smith,2012-09-26 18:08:45


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [38]:
result = run_sql('''SELECT members.firstname, facilities.name
                    FROM bookings
                    INNER JOIN members
                    ON bookings.memid = members.memid
                    INNER JOIN facilities
                    ON bookings.facid = facilities.facid
                    WHERE facilities.name LIKE "Tennis%"
                    ORDER BY members.firstname
                 ''')

# result = run_sql('''
# SELECT * FROM bookings WHERE bookings.facid = 1 OR bookings.facid = 0
# ''')

# result = run_sql('''SELECT * FROM facilities
#                  ''')
result.toPandas()

,firstname,name
0,Anne,Tennis Court 1
1,Anne,Tennis Court 2
2,Anne,Tennis Court 2
3,Anne,Tennis Court 2
4,Anne,Tennis Court 2
5,Anne,Tennis Court 2
6,Anne,Tennis Court 2
7,Anne,Tennis Court 1
8,Anne,Tennis Court 2
9,Anne,Tennis Court 1


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [60]:
result = run_sql('''SELECT facilities.name,
                    IF(bookings.memid = 0, facilities.guestcost * bookings.slots, facilities.membercost * bookings.slots)
                    FROM bookings
                    INNER JOIN facilities
                    ON bookings.facid = facilities.facid
                    WHERE bookings.starttime >= '2012-09-14' 
                    AND bookings.starttime < '2012-09-15'
                    AND IF(bookings.memid = 0, facilities.guestcost * bookings.slots, facilities.membercost * bookings.slots) > 30
                    ORDER BY IF(bookings.memid = 0, facilities.guestcost * bookings.slots, facilities.membercost * bookings.slots) DESC
                 ''')
# result = run_sql('''SELECT * FROM bookings
#                  ''')
result.toPandas()

,name,"(IF((memid = 0), (guestcost * CAST(slots AS DOUBLE)), (membercost * CAST(slots AS DOUBLE))))"
0,Massage Room 2,320.0
1,Massage Room 1,160.0
2,Massage Room 1,160.0
3,Massage Room 1,160.0
4,Tennis Court 2,150.0
5,Tennis Court 1,75.0
6,Tennis Court 1,75.0
7,Tennis Court 2,75.0
8,Squash Court,70.0
9,Massage Room 1,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [61]:
result = run_sql('''
                    SELECT
                    CONCAT(firstname, ' ', surname) AS member,
                    name AS facility,
                    cost
                    FROM
                    (SELECT
                    firstname,
                    surname,
                    name,
                    CASE WHEN firstname = 'GUEST' 
                    THEN guestcost * slots 
                    ELSE membercost * slots 
                    END AS cost,
                    starttime
                    FROM members
                    INNER JOIN bookings
                    ON members.memid = bookings.memid
                    INNER JOIN facilities
                    ON bookings.facid = facilities.facid) AS inner_table
                    WHERE starttime >= '2012-09-14' 
                    AND starttime < '2012-09-15'
                    AND cost > 30
                    ORDER BY cost DESC
                    
                 ''')
result.toPandas()

,member,facility,cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Massage Room 1,160.0
3,GUEST GUEST,Massage Room 1,160.0
4,GUEST GUEST,Tennis Court 2,150.0
5,GUEST GUEST,Tennis Court 1,75.0
6,GUEST GUEST,Tennis Court 1,75.0
7,GUEST GUEST,Tennis Court 2,75.0
8,GUEST GUEST,Squash Court,70.0
9,Jemima Farrell,Massage Room 1,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [62]:
result = run_sql('''
                    SELECT
                    name,
                    revenue
                    FROM
                    (SELECT
                    name,
                    SUM(CASE WHEN memid = 0 
                    THEN guestcost * slots 
                    ELSE membercost * slots 
                    END) AS revenue
                    FROM bookings 
                    INNER JOIN facilities
                    ON bookings.facid = facilities.facid
                    GROUP BY name) AS inner_table
                    WHERE revenue < 1000
                    ORDER BY revenue
                 ''')
# result = run_sql('''SELECT * FROM bookings
#                  ''')
result.toPandas()

,name,revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
